# Assignment 3: Predictive analysis - Part 2: Classification
## Group 105
- Natasa Bolic (300241734)
- Brent Palmer (300193610)
## Imports

In [1]:
# imports
import pandas as pd
import re

## Introduction

## Dataset Description

**Url:** https://www.kaggle.com/datasets/blastchar/telco-customer-churn <br>
**Name:** Telco Customer Churn <br>
**Author:** The dataset was uploaded to `Kaggle` by a user called `BlastChar`, however `IBM` created the original dataset. <br>
**Purpose:** The `Telco Customer Churn` dataset is a synthetic dataset created by `IBM`. The dataset includes data on 7043 customers of the
fictitious telecommunications company `Telco`. For each customer, data is provided regarding their demographics, their account information, 
and the services they have signed up for. The purpose of the dataset is to use the provided customer data to predict customer churn. A column is provided, 
titled `Churn`, that indicates whether or not the customer left within the last month. Since the churn data is provided, we can easily compare
our classification results against the true values. <br>
**Shape:** There are 7043 rows and 21 columns. (7043, 21)<br>
**Features:** 
- `customerID` (categorical): A unique ID provided to each customer in the form `1234-ABCDE`.
- `gender` (categorical): The gender of the customer (includes `Male` and `Female`).
- `SeniorCitizen` (categorical): Indicates if the customer is a senior.
    - `0`: The customer is not a senior.
    - `1`: The customer is a senior.
- `Partner` (categorical): Indicates if the customer has a partner.
    - `Yes`: The customer has a partner.
    - `No`: The customer does not have a partner. 
- `Dependents` (categorical): Indicates if the customer has dependents.
    - `Yes`: The customer has dependents.
    - `No`: The customer does not have dependents. 
- `tenure` (numerical): The number of months the customer has been with the company [0,72].
- `PhoneService` (categorical): Indicates if the customer has a phone service.
    - `Yes`: The customer has a phone service.
    - `No`: The customer does not have a phone service.
- `MultipleLines` (categorical): Indicates if the customer has multiple lines.
    - `Yes`: The customer has multiple lines.
    - `No`: The customer has one line.
    - `No phone service`: The customer does not have phone service.
- `InternetService` (categorical): The customer's internet connection method.
    - `Fiber optic`: The customer's internet connection uses fiber optic technology.
    - `DSL`: The customer's internet connection uses digital subscriber line (DSL) technology.
    - `No`: The customer does not have internet.
- `OnlineSecurity` (categorical): Indicates if the customer has online security.
    - `Yes`: The customer has online security.
    - `No`: The customer does not have online security.
    - `No internet service`: The customer does not have internet service.
- `OnlineBackup` (categorical): Indicates if the customer has online backup.
    - `Yes`: The customer has online backup.
    - `No`: The customer does not have online backup.
    - `No internet service`: The customer does not have internet service. 
- `DeviceProtection` (categorical): Indicates if the customer has device protection.
    - `Yes`: The customer has device protection.
    - `No`: The customer does not have device protection.
    - `No internet service`: The customer does not have internet service. 
- `TechSupport` (categorical): Indicates if the customer has tech support.
    - `Yes`: The customer has tech support.
    - `No`: The customer does not have tech support.
    - `No internet service`: The customer does not have internet service.
- `StreamingTV` (categorical): Indicates if the customer has streaming TV.
    - `Yes`: The customer has streaming TV.
    - `No`: The customer does not have streaming TV.
    - `No internet service`: The customer does not have internet service.
- `StreamingMovies` (categorical): Indicates if the customer has streaming movies.
    - `Yes`: The customer has streaming movies.
    - `No`: The customer does not have streaming movies.
    - `No internet service`: The customer does not have internet service.
- `Contract` (categorical): The contract term of the customer.
    - `Month-to-month`: The customer pays month-to-month.
    - `One year`: The customer has a one-year contract.
    - `Two year`: The customer has a two-year contract. 
- `PaperlessBilling` (categorical): Indicates if the customer has paperless billing.
    - `Yes`: The customer has paperless billing.
    - `No`: The customer does not have paperless billing.
- `PaymentMethod` (categorical): The customer's payment method.
    - `Electronic check`: The customer pays by electronic check.
    - `Mailed check`: The customer pays by mailed check.
    - `Bank transfer (automatic)`: The customer pays automatically by bank transfer.
    - `Credit card (automatic)`: The customer pays automatically by credit card.
- `MonthlyCharges` (numerical): The amount charged to the customer each month [18.25, 118.75]. 
- `TotalCharges` (numerical): The total amount charged to the customer [0.0, 999.9].
- `Churn` (categorical): Indicates if the customer churned, which means if the customer cancelled their service this month.
    - `Yes`: The customer churned.
    - `No`: The customer did not churn. 

## Loading Data and Basic Exploration

In [2]:
# Read in the dataset from a public repository
url = "https://raw.githubusercontent.com/Natasa127/CSI4142-A3/refs/heads/main/Telco-Customer-Churn.csv"
telco_df = pd.read_csv(url)
telco_df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
telco_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [4]:
telco_df.shape

(7043, 21)

## Classification Empirical Study

### (a) Cleaning the data

In this section, we will use the validity checks from `Assignment 2` to determine if the data needs to be cleaned. If needed, an imputation method may be used to clean the data. Note that we will be presenting each of the ten validity checks, although they may be in a different order than in assignment 2. They will be clearly enumerated. Furthermore, in `Assignment 2`, we saved a copy of the invalid rows in our validity checks. In this assignment, we will first check the count of invalid rows; if there are none, we will omit the code that saves a copy, and displays invalid rows. If we find the error count to be non-zero, we will then clean it instead of just saving and displaying the rows.

We have applied every check to every feature that makes sense for that check.

#### Validity Check 1: Exact Duplicates

We will first check for exact duplicates in the dataset. This check verifies that there are no rows that are identical over all columns.

**References:** <br>
Exact Duplicates: https://uottawa.brightspace.com/d2l/le/content/490358/viewContent/6620388/View (Slide 27)

In [5]:
# Exact duplicates check

# Apply the .duplicated method to the DataFrame to create a Series, with exact duplicates set to True
duplicates = telco_df.duplicated()

# Print the number of rows that are exact duplicates
print(f"Number of duplicate rows: {duplicates.sum()}\n")

Number of duplicate rows: 0



Since there are no exact duplicates, no cleaning is required.

#### Validity Check 2: Near Duplicate Errors

In `Assignment 2`, our near duplicate error checker checked if a row differed only by a synonym in a designated attribute. However, when developing our dataset description, we used `.value_counts()` on each categorical attribute, and we noted that there are no synonyms that would cause this error. Thus, we will be using a different type of near duplicate check in this assignment. Instead, we will be checking if any customer's have the exact same information, differing only by their `customerID`.

**References:** <br>
Omit a Column: https://sparkbyexamples.com/pandas/pandas-select-all-columns-except-one-column-in-dataframe/#:~:text=Select%20All%20Except%20One%20Column%20Using%20drop()%20Method%20in,axis%3D1%20or%20columns%20param.

In [6]:
# Near duplicates check

# Apply the .duplcated method to the DataFrame, excluding the customerID column, to create a Series, with exact duplicates of the remaining columns set to True
no_customer_id_duplicates = telco_df.loc[:, telco_df.columns != "customerID"].duplicated()

# Print the number of rows that are near duplicates
print(f"Number of near duplicate rows: {no_customer_id_duplicates.sum()}\n")

Number of near duplicate rows: 22



Since there are 22 near duplicate rows, let us further investigate the actual rows to determine how to handle them.

In [7]:
# Save the invalid rows
invalid_near_duplicate_df = telco_df.loc[no_customer_id_duplicates]

# Display the first 3 rows that are near duplicates
print("Examples of three near duplicate rows:")
invalid_near_duplicate_df.head(3)

Examples of three near duplicate rows:


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
964,9117-SHLZX,Male,0,No,No,1,Yes,No,DSL,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,45.70,45.7,Yes
1338,1934-SJVJK,Male,0,No,No,1,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,20.15,20.15,Yes
1491,8605-ITULD,Female,0,No,No,1,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Month-to-month,No,Mailed check,19.55,19.55,No


We believe that despite our efforts to alter our near duplicate checker to identify rows that vary only by `customerID`, these rows **should not be removed.** There are a few reasons for this:

- There is no row that occurs more than 3 times; in essence, there is no row that occurs a completely unreasonable number of times (there is only one row that occurs three times, and the rest occur twice).
- Given that there are 7043 customers, 22 customers having the same information is plausible. There are a couple reasons why.
    - This is a relatively small portion of the overall sample.
    - Unlike the example in the course slides of near duplicates, we do not have a column like `Name` that would be highly likely to be distinct. All of the columns are reasonably likely to overlap.
        - There are not many options for each column. All of the categorical values only provide a few options, and in many cases the same value would occur across multiple columns (ex: `No internet service` would be shared across 6 columns, so it effectively reduces the number of columns that could be a differentiator). Even the numerical columns are not particularly surprising to overlap—since they represent the prices of services, and presumably `Telco` offers a discrete set of services, the minimal duplication is expected.
    - We analyzed all 22 rows that are duplicates, and the values are values that are logically more likely to be repeated; for example, all the customers had a tenure of 1 month, have a month-to-month plan, no dependents, no partner, were (mostly) not seniors, did not have multiple lines, and most of them do not purchase the internet service. Even of those that did purchase the internet service, none of them had any of the packages creating an unlikely combination. These are all relatively common values, and the internet service specifically reduces the number of differentiating columns. Thus, these users with a common service can only be differentiated by demographic information, their payment methods, and their churn, and it is likely that in some cases this would overlap.
        - As an illustrative example, if there were 15 people with a tenure of exactly 42 months, and had a very specific combination of internet packages, then it would be more likely they were duplicates.
        - To analyze the values of near duplicates, change `invalid_exact_duplicate_df.head(3)` to `invalid_exact_duplicate_df.head(22)`. We have left only 3 examples for brevity.

Thus, we have decided not to remove the rows.

#### Validity Check 3: Format errors

We will use the format check from `Assignment 2` to verify if the `customerID` is in the correct format. The only change necessary is to change the regex to verify that the `customerID` is in the format `1234-ABCDE`. None of the other features have a specific format to follow.

**References:** <br>
Regex Rules: https://www.geeksforgeeks.org/perl-regex-cheat-sheet/

In [8]:
# Format check

# Attribute selection
test_attribute = 'customerID'

# We will fix the regex for the customerID format (1234-ABCDE)
format_regex = r"^\d{4}-[A-Z]{5}$"

# Evaluates a single value against a given regex format
def format_filter(value, format_regex):
    return False if not isinstance(value, str) else bool(re.findall(format_regex, value))

# Apply the function to the test attribute, setting invalid formats to True
invalid_format = telco_df[test_attribute].apply(
    lambda attribute: not format_filter(attribute, format_regex)
)

# Print the number of rows with invalid formatting on the chosen test attribute
print(f"Number of rows where the {test_attribute} value has an invalid format: {invalid_format.sum()}\n")

Number of rows where the customerID value has an invalid format: 0



Since there are no rows with an invalid `customerID` format, no cleaning is required.

#### Validity Check 4: Uniqueness errors

We will use the uniqueness check from `Assignment 2` to validate that the `customerID` values are unique. Minimal changes are necessary to the code. We only need to provide the `customerID` feature name, and change the `sales` DataFrame to `telco_df`. None of the other features are intended to be unique.

In [9]:
# Uniqueness check

# Attribute selection
test_attributes = "customerID"

# Store a series of the counts of each value in the chosen column
attribute_series_counts = telco_df[test_attribute].value_counts()

# Evaluates a single value, checking if it is unique in the chosen column
def uniqueness_filter(value, counts):
    if pd.isna(value):
        return False
    if counts[value] == 1:
        return True
    return False

# Apply the function to the test attribute, setting rows with non-unique values in the designated column to True
invalid_uniqueness = telco_df[test_attribute].apply(
    lambda attribute: not uniqueness_filter(attribute, attribute_series_counts)
)

# Print the number of rows with a value that is not unique in the chosen column
print(f"Number of rows where the {test_attribute} value is not unique: {invalid_uniqueness.sum()}\n")

Number of rows where the customerID value is not unique: 0



Since every `customerID` is unique, no cleaning is required.

#### Validity Check 5: Presence check

We will use the presence check from `Assignment 2` to validate that there is no missing information in any of the columns. Minimal changes are necessary to the code. Instead of taking a parameter input, we will simply iterate over all of the columns to determine if any of them are missing values. We also added a check for strings of whitespace.

In [10]:
# Presence check

# Iterate over each column and perform the presence check on each
for column in telco_df:
    # Apply pd.isna() to the test attribute, or check if it is a string of whitespace, setting rows with missing values in the designated column to True
    invalid_presence = telco_df[column].apply(lambda attribute: (pd.isna(attribute) or (isinstance(attribute, str) and attribute.strip() == "")))

    # Print the number of rows with a missing value in the chosen test attribute
    print(f"Number of rows where the {column} value is missing: {invalid_presence.sum()}")

Number of rows where the customerID value is missing: 0
Number of rows where the gender value is missing: 0
Number of rows where the SeniorCitizen value is missing: 0
Number of rows where the Partner value is missing: 0
Number of rows where the Dependents value is missing: 0
Number of rows where the tenure value is missing: 0
Number of rows where the PhoneService value is missing: 0
Number of rows where the MultipleLines value is missing: 0
Number of rows where the InternetService value is missing: 0
Number of rows where the OnlineSecurity value is missing: 0
Number of rows where the OnlineBackup value is missing: 0
Number of rows where the DeviceProtection value is missing: 0
Number of rows where the TechSupport value is missing: 0
Number of rows where the StreamingTV value is missing: 0
Number of rows where the StreamingMovies value is missing: 0
Number of rows where the Contract value is missing: 0
Number of rows where the PaperlessBilling value is missing: 0
Number of rows where th

There are only missing values in the `TotalCharges` feature. We will investigate this further, first by printing out some examples of invalid rows.

In [11]:
# Extract the rows with a missing TotalCharges
invalid_presence = telco_df["TotalCharges"].apply(lambda attribute: (pd.isna(attribute) or (isinstance(attribute, str) and attribute.strip() == "")))

# Save the invalid rows
invalid_presence_df = telco_df.loc[invalid_presence]

# Display the first 3 rows with a missing value
print(f"Examples of three rows where the TotalCharges value is missing:")
invalid_presence_df.head(3)

Examples of three rows where the TotalCharges value is missing:


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
488,4472-LVYGI,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,...,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,,No
753,3115-CZMZD,Male,0,No,Yes,0,Yes,No,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,,No
936,5709-LVOEQ,Female,0,Yes,Yes,0,Yes,No,DSL,Yes,...,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,,No


Note that all of the rows that are missing a `TotalCharges` value have a `tenure` of 0. This makes sense; since this is the customer's first month, they have not been charged yet.

In [12]:
telco_df["tenure"].value_counts()[0]

np.int64(11)

There are 11 customers with a `tenure` of zero, corresponding to the number of customers with a missing `TotalCharges` value. Thus, this must be the reason. Let us replace the missing values with a value of 0, as from a logical standpoint, these customers have been charged $0 so far. We will make this change in a copy of the dataset called `cleaned_telco_df`. Any subsequent cleaning will be done using this dataset.

In [13]:
cleaned_telco_df = telco_df.copy()
cleaned_telco_df.loc[cleaned_telco_df["tenure"] == 0, "TotalCharges"] = 0

# Extract the rows with a missing TotalCharges
invalid_presence = cleaned_telco_df["TotalCharges"].apply(lambda attribute: (pd.isna(attribute) or (isinstance(attribute, str) and attribute.strip() == "")))

# Verify that there are no longer missing values
print(f"Number of rows where the TotalCharges value is missing: {invalid_presence.sum()}")

Number of rows where the TotalCharges value is missing: 0


Now, we have solved the presence error problem by imputing the value of 0 in `cleaned_telco_df`, which is done since the missing values correspond to customers who have not been charged yet.

#### Validity Check 6: Look-up Check

We will use the look-up check from `Assignment 2` to validate that select features' values exist in a corresponding pre-defined finite set of values. Minimal changes are necessary to the code. Instead of taking a parameter input, we define a dictionary containing the relevant features as the keys, and their respective look-up tables as values. Then, we iterate over all of these features and check if their values are included in their look-up tables.

In [14]:
# Look-up check

# Dictionary containing every feature's look-up table of valid values
look_up_dict = {
    "gender": ["Male", "Female"],
    "SeniorCitizen": [0, 1],
    "Partner": ["Yes", "No"],
    "Dependents": ["Yes", "No"],
    "PhoneService": ["Yes", "No"],
    "MultipleLines": ["Yes", "No", "No phone service"],
    "InternetService": ["Fiber optic", "DSL", "No"],
    "OnlineSecurity": ["Yes", "No", "No internet service"],
    "OnlineBackup": ["Yes", "No", "No internet service"],
    "DeviceProtection": ["Yes", "No", "No internet service"],
    "TechSupport": ["Yes", "No", "No internet service"],
    "StreamingTV": ["Yes", "No", "No internet service"],
    "StreamingMovies": ["Yes", "No", "No internet service"],
    "Contract": ["Month-to-month", "One year", "Two year"],
    "PaperlessBilling": ["Yes", "No"],
    "PaymentMethod": ["Electronic check", "Mailed check", "Bank transfer (automatic)", "Credit card (automatic)"],
    "Churn": ["Yes", "No"]
}

# Checks if a single value is in the look-up table
def look_up_filter(value, look_up_table):
    return value in look_up_table

# Apply the function to every feature, setting rows whose value is not in the look-up table to True
for feature in look_up_dict.keys():
    invalid_look_up = cleaned_telco_df[feature].apply(
        lambda attribute: not look_up_filter(attribute, look_up_dict[feature])
    )
    # Print the number of rows with a value that is not in the look-up table for the designated attribute
    print(f"Number of rows where the {feature} value is not in the look-up table: {invalid_look_up.sum()}")

Number of rows where the gender value is not in the look-up table: 0
Number of rows where the SeniorCitizen value is not in the look-up table: 0
Number of rows where the Partner value is not in the look-up table: 0
Number of rows where the Dependents value is not in the look-up table: 0
Number of rows where the PhoneService value is not in the look-up table: 0
Number of rows where the MultipleLines value is not in the look-up table: 0
Number of rows where the InternetService value is not in the look-up table: 0
Number of rows where the OnlineSecurity value is not in the look-up table: 0
Number of rows where the OnlineBackup value is not in the look-up table: 0
Number of rows where the DeviceProtection value is not in the look-up table: 0
Number of rows where the TechSupport value is not in the look-up table: 0
Number of rows where the StreamingTV value is not in the look-up table: 0
Number of rows where the StreamingMovies value is not in the look-up table: 0
Number of rows where the C

Since none of the features have a row with a value not in the corresponding look-up table, no cleaning is required.

#### Validity Check 7: Consistency Check

We will use the consistency check from `Assignment 2` to validate that data in a row follows some designated rule that involves multiple columns. Minimal changes are necessary to the code. We simply specify an array of two different rules we want to apply to our dataset, and iterate over them to perform the validation. There are three consistency checks that make sense for this dataset:
1. If a user does not have the phone service (value of `No` for `PhoneService`), then they must have a value of `No phone service` in the `MultipleLines` feature. Conversely, those with phone service (value of `Yes` for `PhoneService`) must not have a value of `No phone service` in the `MultipleLines` feature.
2. If a user does not have the internet service (values of `No` for `InternetService`), then they must have a value of `No internet service` in the `OnlineSecurity`, `OnlineBackup`, `DeviceProtection`, `TechSupport`, `StreamingTV`, and `StreamingMovies` features. Conversely, those with the internet service (value of `Fiber optic` or `DSL` for `InternetService`) must not have a value of `No internet service` in the `OnlineSecurity`, `OnlineBackup`, `DeviceProtection`, `TechSupport`, `StreamingTV`, and `StreamingMovies` features.
3. If a user has a `tenure` of `0`, then their `TotalCharges` must be `0`. Otherwise, their `TotalCharges` must be greater than `0`.

**References:** <br>
Eval: https://docs.python.org/3/library/functions.html#eval

In [15]:
# Consistency check

# An array containing the three rules described above
rules = [
    (
        '(row["InternetService"] == "No" and row["OnlineSecurity"] == row["OnlineBackup"] == row["DeviceProtection"]' 
        '== row["TechSupport"] == row["StreamingTV"] == row["StreamingMovies"] == "No internet service") or '
        '(row["InternetService"] in ["Fiber optic", "DSL"] and row["OnlineSecurity"] in ["Yes", "No"] and '
        'row["OnlineBackup"] in ["Yes", "No"] and row["DeviceProtection"] in ["Yes", "No"] and row["TechSupport"] in ["Yes", "No"] and '
        'row["StreamingTV"] in ["Yes", "No"] and row["StreamingMovies"] in ["Yes", "No"])'
    ),
    '(row["PhoneService"] == "No" and row["MultipleLines"] == "No phone service") or (row["PhoneService"] == "Yes" and row["MultipleLines"] in ["Yes", "No"])',
    '(row["tenure"] == 0 and row["TotalCharges"] == 0) or (row["tenure"] > 0 and row["TotalCharges"] > 0)'
]

# Evalutes a single row against a given rule
def consistency_filter(row, rule):
    try:
        # Convert the row to a dictionary, since eval() can use a dictionary
        row = row.to_dict()

        # Convert the numeric attributes to float to allow for expression evaluation
        for key, value in row.items():
            try:
                row[key] = float(value)
            except Exception as e:
                pass

        # Return the result of the expression
        return eval(rule)
    except Exception as e:
        # If the eval fails (for example, 2.0 * "error" will throw an error), return False
        return False

# Validate each rule
for rule in rules:
    # Apply the function to each row, setting inconsistent rows to True
    invalid_consistency = cleaned_telco_df.apply(
        lambda row: not consistency_filter(row, rule),
        axis=1
    )
    
    # Print the number of inconsistent rows based on the provided rule
    print(f"Number of rows where the rule {rule} is invalid: {invalid_consistency.sum()}\n")

Number of rows where the rule (row["InternetService"] == "No" and row["OnlineSecurity"] == row["OnlineBackup"] == row["DeviceProtection"]== row["TechSupport"] == row["StreamingTV"] == row["StreamingMovies"] == "No internet service") or (row["InternetService"] in ["Fiber optic", "DSL"] and row["OnlineSecurity"] in ["Yes", "No"] and row["OnlineBackup"] in ["Yes", "No"] and row["DeviceProtection"] in ["Yes", "No"] and row["TechSupport"] in ["Yes", "No"] and row["StreamingTV"] in ["Yes", "No"] and row["StreamingMovies"] in ["Yes", "No"]) is invalid: 0

Number of rows where the rule (row["PhoneService"] == "No" and row["MultipleLines"] == "No phone service") or (row["PhoneService"] == "Yes" and row["MultipleLines"] in ["Yes", "No"]) is invalid: 0

Number of rows where the rule (row["tenure"] == 0 and row["TotalCharges"] == 0) or (row["tenure"] > 0 and row["TotalCharges"] > 0) is invalid: 0



Since none of the rows are inconsistent for any of the rules, no cleaning is required.

#### Validity Check 8: Data Type Check

We will use the data type check from `Assignment 2` to validate that the data stored is the correct data type. We can apply this to every column. We will use our method 1 from `Assignment 2`, which simply checks if the data in its current format has the desired data type. Minimal changes are necessary to the code. We simply specify a dictionary, where the keys are the features, and the values are the desired data types. We iterate over each feature in the dictionary, applying the data type check to each.

In [16]:
# Data type check

# Evalutes a single value's data type against the desired data type
def type_filter_method1(value, test_datatype):
    if pd.isna(value):
        return False
    return isinstance(value, test_datatype)

# Create a dictionary that maps each attribute to its correct datatype
data_type_dict = {
    "customerID": str,
    "gender": str,
    "SeniorCitizen": int,
    "Partner": str,
    "Dependents": str,
    "tenure": int,
    "PhoneService": str,
    "MultipleLines": str,
    "InternetService": str,
    "OnlineSecurity": str,
    "OnlineBackup": str,
    "DeviceProtection": str,
    "TechSupport": str,
    "StreamingTV": str,
    "StreamingMovies": str,
    "Contract": str,
    "PaperlessBilling": str,
    "PaymentMethod": str,
    "MonthlyCharges": float,
    "TotalCharges": float,
    "Churn": str
}

# Apply the function to every feature, setting rows whose value is not stored as the correct datatype to True
for feature in data_type_dict.keys():
    invalid_datatype = cleaned_telco_df[feature].apply(
        lambda attribute: not type_filter_method1(attribute, data_type_dict[feature])
    )
    # Print the number of rows with a value that is not stored as the correct datatype for the designated attribute
    print(f"Number of rows where the {feature} value is not stored as the correct datatype ({data_type_dict[feature]}): {invalid_datatype.sum()}")

Number of rows where the customerID value is not stored as the correct datatype (<class 'str'>): 0
Number of rows where the gender value is not stored as the correct datatype (<class 'str'>): 0
Number of rows where the SeniorCitizen value is not stored as the correct datatype (<class 'int'>): 0
Number of rows where the Partner value is not stored as the correct datatype (<class 'str'>): 0
Number of rows where the Dependents value is not stored as the correct datatype (<class 'str'>): 0
Number of rows where the tenure value is not stored as the correct datatype (<class 'int'>): 0
Number of rows where the PhoneService value is not stored as the correct datatype (<class 'str'>): 0
Number of rows where the MultipleLines value is not stored as the correct datatype (<class 'str'>): 0
Number of rows where the InternetService value is not stored as the correct datatype (<class 'str'>): 0
Number of rows where the OnlineSecurity value is not stored as the correct datatype (<class 'str'>): 0
Numb

The only feature with an issue is `TotalCharges`. It should be stored as a `float`, but it is stored as an `object`. This is expected, as we noted earlier that the 11 customer's that had a `tenure` of `0` were initially stored as empty strings instead of `0`. Since we have already converted the empty strings to `0`, we can simply cast the values to the `float` datatype, since the `objects` hold `float` values.

**References:** <br>
Cast a Column: https://stackoverflow.com/questions/48094854/pandas-convert-data-type-from-object-to-float

In [18]:
# Cast the TotalCharges feature to the float data type
cleaned_telco_df["TotalCharges"] = cleaned_telco_df.TotalCharges.astype(float)

# Rerun the validity check to verify the cleaning worked.

invalid_datatype = cleaned_telco_df["TotalCharges"].apply(
    lambda attribute: not type_filter_method1(attribute, float)
)

# Print the number of rows with a value that is not stored as the correct datatype for the designated attribute
print(f"Number of rows where the TotalCharges value is not stored as the correct datatype (float): {invalid_datatype.sum()}")

Number of rows where the TotalCharges value is not stored as the correct datatype (float): 0


Now, we have solved the data type error problem by casting the `TotalCharges` column to `float` in cleaned_telco_df. The dataset has now been cleaned for the data type check.

#### Validity Check 9: Length Check

We will use the length check from `Assignment 2` to validate that the length of the categorical values is within a designated range. Minimal changes are necessary to the code. Instead of taking a parameter input, we define a dictionary containing the relevant features as the keys, and their respective minimum and maximum lengths as values (stored as a tuple, (`min`, `max`)). Then, we iterate over all of these features and check if their length values are within the designated range.

Our minimum and maximum length values are based on the minimum and maximum lengths of the valid values. For example, for `Partner`, the valid values are `Yes` and `No`, so we have set the minimum valid length to `2` and the maximum valid length to `3` accordingly.

In [44]:
# Length check

# Dictionary containing every feature's minimum and maximum valid lengths
length_dict = {
    "customerID": (10, 10), # All customerIDs are exactly 10 characters
    "gender": (4, 6), # Male, Female
    "Partner": (2, 3), # No, Yes
    "Dependents": (2, 3), # No, Yes
    "PhoneService": (2, 3), # No, Yes
    "MultipleLines": (2, 16), # No, No phone service
    "InternetService": (2, 11), # No, Fiber optic
    "OnlineSecurity": (2, 19), # No, No internet service
    "OnlineBackup": (2, 19), # No, No internet service
    "DeviceProtection": (2, 19), # No, No internet service
    "TechSupport": (2, 19), # No, No internet service
    "StreamingTV": (2, 19), # No, No internet service
    "StreamingMovies": (2, 19), # No, No internet service
    "Contract": (8, 14), # One year, Month-to-month
    "PaperlessBilling": (2, 3), # No, Yes
    "PaymentMethod": (12, 25), # Mailed check, Bank transfer (automatic)
    "Churn": (2, 3) # No, Yes
}

# Evaluates a single value's length against the given valid range of lengths
def length_filter(value, minimum_length, maximum_length):
    if pd.isna(value) or not isinstance(value, str):
        return False
    return minimum_length <= len(value) <= maximum_length

# Apply the function to every feature in the dictionary, setting invalid length values to True
for feature in length_dict.keys():
    # Apply the function to the test attribute, setting out of range length values to True
    invalid_length = cleaned_telco_df[feature].apply(
        lambda attribute: not length_filter(attribute, length_dict[feature][0], length_dict[feature][1])
    )

    # Print the number of rows with a length value outside of the given valid length range for the designated attribute
    print(f"Number of rows where the {feature} value's length is outside of the defined range of valid lengths {length_dict[feature]}: {invalid_length.sum()}")


Number of rows where the customerID value's length is outside of the defined range of valid lengths (10, 10): 0
Number of rows where the gender value's length is outside of the defined range of valid lengths (4, 6): 0
Number of rows where the Partner value's length is outside of the defined range of valid lengths (2, 3): 0
Number of rows where the Dependents value's length is outside of the defined range of valid lengths (2, 3): 0
Number of rows where the PhoneService value's length is outside of the defined range of valid lengths (2, 3): 0
Number of rows where the MultipleLines value's length is outside of the defined range of valid lengths (2, 16): 0
Number of rows where the InternetService value's length is outside of the defined range of valid lengths (2, 11): 0
Number of rows where the OnlineSecurity value's length is outside of the defined range of valid lengths (2, 19): 0
Number of rows where the OnlineBackup value's length is outside of the defined range of valid lengths (2, 19

Since none of the values fail the length check, no cleaning is required.

#### Validity Check 10: Range Check

We will use the range check from `Assignment 2` to validate that the values of the numerical values are within the minimum and maximum acceptable values. Minimal changes are necessary to the code. Instead of taking a parameter input, we define a dictionary containing the relevant features as the keys, and their respective minimum and maximum values as values (stored as a tuple, (`min`, `max`)). Then, we iterate over all of these features and check if their values are within the designated range.

The following is a rationale for each of the ranges provided:
- `tenure` [0, 900]: `Telco` is a fictitious company, so there is no real founding date. Let us suppose that `Telco` is a legacy telecommunications company, established in 1950. Then, the maximum `tenure` possible would be `900` months. A `tenure` cannot be negative, so the minimum is `0`, representing customers in their first month of service.
- `MonthlyCharges` [5, 297]: Since `Telco` is a fictious company, there is not a set of services that can be checked to determine minimum and maximum values. From some industry research, we found a company that offers a home phone service for `5` dollars, so we will set the minimum monthly charge to `5` dollars. We are setting the maximum based on Bell's current maximum home phone price plus Bell's maximum internet service price, which is (`137` (Home Phone with Five Lines) + `160` (Gigabit Fibe 8.0)). Technically customers could buy more than five phone lines, but this estimate should be sufficient for this assignment.
- `TotalCharges` [0, 267300]: The minimum `TotalCharges` is the minimum `tenure` multiplied by the minimum `MonthlyCharges`, which is `0` due to a possible `tenure` of `0`. The maximum `TotalCharges` is based on the maximum `tenure` multiplied by the maximum `MonthlyCharges`, which is `267300`.

**References:** <br>
5-dollar Home Phone: https://www.fongo.com/services/fongo-home-phone/ <br>
Bell Pricing: https://www.bell.ca/

In [47]:
# Range check

# Dictionary containing every feature's minimum and maximum valid values
range_dict = {
    "tenure": (0, 900),
    "MonthlyCharges": (5, 297),
    "TotalCharges": (0, 267300)
}

# Evaluates a single value against a given range
def range_filter(value, minimum, maximum):
    try:
        value = float(value)
    except Exception as e:
        return False
    return minimum <= value <= maximum

# Apply the function to every feature in the dictionary, setting out of range values to True
for feature in range_dict.keys():
    invalid_range = cleaned_telco_df[feature].apply(
        lambda attribute: not range_filter(attribute, range_dict[feature][0], range_dict[feature][1])
    )
    # Print the number of rows with a value that is not stored as the correct datatype for the designated attribute
    print(f"Number of rows where the {feature} value is not in the correct range {range_dict[feature]}: {invalid_range.sum()}")

Number of rows where the tenure value is not in the correct range (0, 900): 0
Number of rows where the MonthlyCharges value is not in the correct range (5, 297): 0
Number of rows where the TotalCharges value is not in the correct range (0, 267300): 0


Since none of the values fail the range check, no cleaning is required.

## Conclusion

## References